# Social Computing/Social Gaming - Summer 2020

# Exercise Sheet 3: Collaborative Filtering with Steam Games

In this exercise, we will build a collaborative filtering recommender system using data we gather from Steam. We will use your friends list to get information about owned games for each ID, and the time each game was played.

Usually, collaborative filtering is based on some sort of rating to determine the similarity between users. However, for games, the enjoyment and a rating do not always match. Additionally, only about 10% of players actually rate the games they play, which would make for a very incomplete dataset. Therefore, the playtime will be used instead of a rating system. This has the added benefit that playtime is usually the most authentic metric of enjoyment, as players are very unlikely to spend much time on a game they don't enjoy.

## Task 3.1: Obtaining the data


**1.** Your first task is to gather the data needed to create the recommender system. Create a data structure that holds the needed information for each player and game.  
**Note:** You cannot obtain a list from your profile with the Steam API unless your profile is set to public. 

If you do not have a Steam profile, you can use the default values. 
However, we encourage you to use your own profile. 

**Hint**: To obtain the games a user owns, use this: `games = data['response']['games']`. This returns a list of games, including the playtime (in minutes) which can be retrieved like this: `playtime = game['playtime_forever']` , where game refers to an item from the list of games. 

In [2]:
#Use this if you want to work with the default IDs
import requests
import urllib
import pandas as pd
import json
from urllib.request import Request, urlopen
from pandas.io.json import json_normalize
from requests.exceptions import HTTPError

# You can replace these values with your own ID and API key
key = "CB35B8F8DCE9135DDAA3B0328FCE0103"
id = "76561198329838242"
url = "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key="+key+"&steamids="+id
r = requests.get(url)
data = r.json()

# Get friendslist
request = Request("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+key+"&steamid="+id+"&relationship=friend")
response = urlopen(request)
elevations = response.read()
data = json.loads(elevations)
friendslist = data['friendslist']
friends = friendslist['friends']

friendids =[]
tempIDs = []
for friend in friends:
    friendids.append(friend['steamid'])
    
print(len(friendids))
#get friends of friends:
x = 0
while x < len(friendids):
    friendID = friendids[x]
    request = Request("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+key+"&steamid="+friendID+"&relationship=friend")
    try:
        response = urlopen(request)    
    except urllib.error.HTTPError  as e:
        print('401')
    elevations = response.read()
    try:
        data = json.loads(elevations)
    except json.JSONDecodeError:
        print('couldnt decode')
    friendslist = data['friendslist']
    friends = friendslist['friends']

    friendidsNew =[]
    for friend in friends:
        friendidsNew.append(friend['steamid'])
        
    tempIDs+=friendidsNew
    x+=1

friendids += tempIDs
friendids = list(dict.fromkeys(friendids))
friendids = list(set(friendids))
print(len(friendids))


64
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
6655


In [3]:
# Trim the list of IDs to reasonable values:
if len(friendids)>250:
    friendids = friendids[:250]    
print(len(friendids))

users_gamedicts = {} # The dictionary containing all information for every ID
gamedict = {} # A dict containing information for one player

# Get owned games of friendslist:
request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="+key+"&steamid="+id+"&include_appinfo=1&format=json")

# TODO:
# Open the URL and read the json response and retrieve the games of your friends and their playtime
# Save the games into a dictionary with key=name and values=playtime
# Hint 1: You can obtain the games a user owns with data['response']['games']
# Hint 2: You can retrieve their playtime with game['playtime_forever']
games=[]
friends = friendslist['friends']
friendSteamIds = []
for friend in friends:
    friendSteamIds.append(friend['steamid'])
#assert(len(friendSteamIds)==64)    
for id in friendSteamIds:
    
    request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="+key+"&steamid="+id+"&include_appinfo=1&format=json")
    try:
        response = urlopen(request)
    except urllib.error.HTTPError  as e:
        print('401')
    try:    
        elevations = response.read()
    except json.JSONDecodeError:
        print('couldnt decode')    
    data = json.loads(elevations)
    
    if(data['response'] and data['response']['game_count']>0):
        games = data['response']['games']
        for game in games:
            name = game['name']
            playtime = game['playtime_forever']
            gamedict.update({name:playtime})

        # Add the dictionary to the users_gamedict       
        users_gamedicts[int(id)]=gamedict
        gamedict={}
    

# Do the same for the friends of your friends
for friendID in friendids:
    request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="+key+"&steamid="+friendID+"&include_appinfo=1&format=json")
    try:
        response = urlopen(request)
    except urllib.error.HTTPError  as e:
        print('401')
    try:    
        elevations = response.read()
    except json.JSONDecodeError:
        print('couldnt decode')    
    data = json.loads(elevations)
    
    if(data['response'] and data['response']['game_count']>0):
        games = data['response']['games']
        for game in games:
            name = game['name']
            playtime = game['playtime_forever']
            gamedict.update({name:playtime})

        # Add the dictionary to the users_gamedict       
        users_gamedicts[friendID]=gamedict
        gamedict={}
assert(len(users_gamedicts)<=250)


250


## Task 3.2: Association rule mining

Before we start with the "real" recommender system, let us take a look at a more general form of recommending items using association rules.

The concept of association rule mining is rather simple: Looking at an itemset, one tries to find dependencies between items that could "belong together". A common example would be buying food at the store: If, for example, meat and salt are bought together often, but meat without salt not that often, it is assumed that there is a connection between those two. For games, if it was found that most of the users who own the demo version of a game also own the full version of that game, it would be a reasonable assumption that these users liked the demo and therefore bought the full version.


Let us first cover the mathematical basis for association rules. The most important metrics used are **support**,  **confidence** and **lift**. The first is defined as the amount of times an item occurs in the itemset divided by the total number of items in the set; the second is defined as the support of a list of items [x,y,...] divided by the support of x. Lift is a measure describing the correlation between items. Written down mathematically:

$$supp(x)= \frac{len(x)}{len(n)}$$

$$conf(x=>y) = \frac{supp(x,y)}{supp(x)}$$

$$lift(x=>y) = \frac{P(x \cap y)}{P(x) * P(y)}$$



It is important to note that support refers to an item or a list of items, while confidence refers to a rule. Also note that a lift of 1 means that x and y occur independently of each other, while a lift greater 1 means a positive correlation.


**1.** Your task here is to first convert the dictionary you created into a list of lists as this is the input required for the algorithm to work. Then, print out the most frequent items using the `min_support` attribute. Finally, print out the association rules and play around with the threshold value to get a reasonable amount of rules. 


**2.** Discuss your results and try to answer the following questions: What kind of recommendations can be made? What does a confidence of 1.0 mean and is it meaningful for recommending games? Can you spot a correlation between the games with the highest support and the rules with the highest confidence? How does this affect the lift?  
**Hint:** Play around with the threshold values until you get a reasonable amount (4-30) rows as output.

In [4]:
gamesofallusers = []

# TODO: Convert the gamedict to a list of lists
for key in users_gamedicts:
    values_dict = users_gamedicts[key]
    gamesofallusers.append(list(values_dict.keys()))

assert(len(gamesofallusers)==len(users_gamedicts))    
# Remove common Steam entries that are not games:
for game in gamesofallusers:
    if 'Dota 2 Test' in game:
        game.remove('Dota 2 Test')
    if 'True Sight' in game:
        game.remove('True Sight')
    if 'True Sight: Episode 1' in game:
        game.remove('True Sight: Episode 1')
    if 'True Sight: Episode 2' in game:
        game.remove('True Sight: Episode 2')
    if 'True Sight: Episode 3' in game:
        game.remove('True Sight: Episode 3')
    if 'True Sight: The Kiev Major Grand Finals' in game:
        game.remove('True Sight: The Kiev Major Grand Finals')
    if 'True Sight: The International 2017' in game:
        game.remove('True Sight: The International 2017')
    if 'True Sight: The International 2018 Finals' in game:
        game.remove('True Sight: The International 2018 Finals') 

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
# TODO: Tinker around with the values
te_ary = te.fit(gamesofallusers).transform(gamesofallusers)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.327273,(Amnesia: The Dark Descent)
1,0.309091,(Battlerite)
2,0.309091,(Battlerite Public Test)
3,0.418182,(Borderlands 2)
4,0.454545,(Brawlhalla)
...,...,...
499,0.327273,"(PAYDAY 2, Killing Floor, Left 4 Dead 2, Count..."
500,0.309091,"(Killing Floor, Left 4 Dead 2, PLAYERUNKNOWN'S..."
501,0.345455,"(Portal 2, Killing Floor, Left 4 Dead 2, Count..."
502,0.309091,"(PAYDAY 2, Killing Floor, Left 4 Dead 2, Garry..."


In [5]:
from mlxtend.frequent_patterns import association_rules

# TODO: Play around with the treshold value
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Counter-Strike: Global Offensive),(Borderlands 2),0.891892,0.527027,0.486486,0.545455,1.034965,0.016435,1.040541
1,(Borderlands 2),(Counter-Strike: Global Offensive),0.527027,0.891892,0.486486,0.923077,1.034965,0.016435,1.405405
2,(Borderlands 2),(Garry's Mod),0.527027,0.554054,0.432432,0.820513,1.480926,0.140431,2.484556
3,(Garry's Mod),(Borderlands 2),0.554054,0.527027,0.432432,0.780488,1.480926,0.140431,2.154655
4,(Left 4 Dead 2),(Borderlands 2),0.594595,0.527027,0.418919,0.704545,1.336830,0.105551,1.600832
...,...,...,...,...,...,...,...,...,...
107,"(Rust, Rust - Staging Branch)",(PAYDAY 2),0.500000,0.621622,0.418919,0.837838,1.347826,0.108108,2.333333
108,"(PAYDAY 2, Rust - Staging Branch)",(Rust),0.418919,0.500000,0.418919,1.000000,2.000000,0.209459,inf
109,(Rust),"(PAYDAY 2, Rust - Staging Branch)",0.500000,0.418919,0.418919,0.837838,2.000000,0.209459,3.583333
110,(PAYDAY 2),"(Rust, Rust - Staging Branch)",0.621622,0.500000,0.418919,0.673913,1.347826,0.108108,1.533333


**TODO: Write your observations here:**





## Task 3.3: The Recommender System: Similarity Score


Finally, it is time to build the recommender system. 

**1.** The first thing to do is to implement a similarity score that will be used to predict a user's playtime of an unowned game. We implement a similarity score between two users by taking the relative distance between two players. We use the following formula:

$$d(u, v) = \sum_{i~\in~\textrm{common_games}} \frac{|r_{u,i} - r_{v,i}|}{r_{v,i}}$$  

Where $u$ and $v$ are users and $r_{u,i}$ is the playtime of user $u$ for game $i$. 

You can then return the similarity with  
$$ w_{u,v} = \frac{1}{1 + d(u, v)} $$

**Note:** If no common games exist return 0.

In [93]:
# Here we will calculate the similarity score between two friends based on their common games:
import math
def calculate_similarity(user1ID, user2ID):
    
    # TODO:
    common_games=0
    duv=0
    user1games = users_gamedicts[user1ID]
    user2games = users_gamedicts[user2ID]
    for key in user1games:
        if key in user2games:
            if(user2games[key]!=0 and user1games[key]!=0):
                common_games+=1
                diff = user1games[key] - user2games[key]
                duv+=abs(diff)/user2games[key]
    
    if(common_games==0):
        return 0
    else:
        return 1/(1+duv)
    

## Task 3.4: Recommender System: Predict ratings

With the similarity score calculated, we can now predict a user's playtime for games they don't own.   
**1.** First, we create a set of all games, but we delete all games that are owned by less than 3 players. The reason is simple: If only 1 or 2 players own a game, it is impossible to derive a meaningful prediction since there is not enough data. 

The predicted playtime for a game works analogous to the predicted rating of a movie/item in a conventional collaborative filtering recommender system:

$$r_{u,i} = \frac{\sum_{v \in N_i(u)} w_{u,v}r_{v,i}}{\sum_{v \in N_i(u)} w_{u,v}}$$

where 
- $r_{u,i}$ is the estimated recommendation of item $i$ for target user $u$. 
- $N_i(u)$ is the set of similar users to target user $u$ for the designated item $i$. 
- $w_{u,v}$ is the similarity score between users $u$ and $v$ (used as a weighting factor).  

**Note:** In our case, we use playtime as a recommendation measure and the set $N_i(u)$ consists of user $u$ friends list and friends of friends list. In our scenario, we do not need the index $i$ as our friends list does not change between games.

In [60]:
def default_user_owned_games():
    key = "CB35B8F8DCE9135DDAA3B0328FCE0103"
    id = "76561198329838242"
    request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="+key+"&steamid="+id+"&include_appinfo=1&format=json")
    try:
        response = urlopen(request)
    except urllib.error.HTTPError  as e:
        print('401')
    try:    
        elevations = response.read()
    except json.JSONDecodeError:
        print('couldnt decode')    
    data = json.loads(elevations)

    user_owned_games =[]
    for game in data['response']['games']:
        user_owned_games.append(game['name'])

    return user_owned_games

In [61]:
# List of all games that are owned by at least 1 person:
allGames = []
for user in gamesofallusers:
    for game in user:
        allGames.append(game)
        
# TODO : Create a list of games owned by at least 3 people
allGamesUnique=[]
for game in allGames:
    if allGames.count(game)>=3 and game not in allGamesUnique:
        allGamesUnique.append(game)



# Find out which games you do not own out of all games because we are only interested in recommendations for games that we do not own
def difference(allGames, usersgames): 
    # TODO:
    unowned_games=[]
    for game in allGames:
        if game not in usersgames:
            unowned_games.append(game)

    return unowned_games
    
# Predict ratings based on the formula above for each unowned game
def predict_ratings():
    # TODO:
    '''Hint: Iterate over all unowned games and for each game calculate a rating based on your friends playtime and similarity score ''' 
    unowned_games_rec={}
    user_owned_games=default_user_owned_games();
    user_unowned_games = difference(allGamesUnique,user_owned_games)
    for unowned_game in user_unowned_games:
        num=0
        denom=0
        rui=0
        for friend in users_gamedicts:
            friends_games_dict=users_gamedicts[friend]
            if unowned_game in friends_games_dict.keys():
                similarity = calculate_similarity(76561198329838242,friend)
                rvi = friends_games_dict[unowned_game]
                num+=similarity*rvi
                denom+=similarity
            
        
        if num==0 or denom==0:
            rui = 0
        else:
            rui = num/denom
        unowned_games_rec.update({unowned_game:rui})
    return unowned_games_rec
    
recomm = predict_ratings()

In [62]:
sorted(recomm.items(), key=lambda x:x[1], reverse=True)

[('Oxygen Not Included', 51060.0),
 ('Counter-Strike: Global Offensive', 44895.57820187358),
 ('Crusaders of the Lost Idols', 43871.98706433573),
 ('Enter the Gungeon', 20485.578081004453),
 ("Tom Clancy's Rainbow Six Siege", 19828.020470034087),
 ('Warframe', 19393.97981699013),
 ('Terraria', 17676.652914157123),
 ('Idle Champions of the Forgotten Realms', 15939.548288505894),
 ('PAYDAY 2', 13816.206913534023),
 ('Plants vs. Zombies: Game of the Year', 12765.153977911194),
 ('Hearts of Iron IV', 11477.309270053247),
 ('Total War: ATTILA', 11148.811661312062),
 ('RIFT', 10790.682549980755),
 ('The Binding of Isaac: Rebirth', 10319.537321555186),
 ('eFootball PES 2020', 10005.52583862194),
 ('Rust', 9908.774470844795),
 ('DARK SOULS™ III', 9777.523070713782),
 ('The Binding of Isaac', 9564.087489057525),
 ('Fallout 4', 9034.681859220265),
 ('Call of Duty: Black Ops III', 8846.440621856173),
 ('AdVenture Communist', 8404.958111245005),
 ('Grand Theft Auto V', 8084.615297698517),
 ('Starb

In [46]:
users_gamedicts[76561198329838242]

{'Path of Exile': 16353,
 'Europa Universalis IV': 113212,
 'Creativerse': 0,
 'Titan Quest Anniversary Edition': 10354,
 'Black Desert Online': 3697,
 'Crusader Kings II': 5896}

In [54]:

        for friend in users_gamedicts:
            friends_games_dict=users_gamedicts[friend]
            if 'Arma 3' in friends_games_dict.keys():
                similarity = calculate_similarity(76561198329838242,friend)
                if similarity>0:
                    print(friend)
                rvi = friends_games_dict['Arma 3']
                num+=similarity*rvi
                denom+=similarity    
        rui = num/denom
        unowned_games_rec.update({unowned_game:rui})


76561198026230927
76561198797326298
76561198108479145
76561197999166655
76561198111504494
76561198043051688
76561198050025217
76561198027956483


In [49]:
(default_user_owned_games())

['Path of Exile',
 'Europa Universalis IV',
 'Creativerse',
 'Titan Quest Anniversary Edition',
 'Black Desert Online',
 'Crusader Kings II']

In [101]:
for key in default_user_owned_games():
    if key in users_gamedicts['76561198027956483'].keys():
        games = users_gamedicts['76561198027956483']
        print(key,games[key])
        print("Arma 3",games['Arma 3'])

Path of Exile 1058
Arma 3 201
Crusader Kings II 11
Arma 3 201


In [100]:
calculate_similarity(76561198329838242,'76561198027956483')

0.0018166739070336877

## Task 3.5: Recommender System: Discussion

**1.** Sort the predicted ratings by estimated playtime (highest first) and print out the top 5 predictions for you (or the default user if you are using the default ID). 

**2.** Discuss the difference in recommendations between the collaborative filtering approach and the association rule approach. Would you consider one more accurate than the other? Why/why not?

In [79]:
# TODO 1: 



**TODO 2: Write your observations here:**
